The below cell should be run first. This loads the required Python packages and defines the relevant functions. For further information please see DOI: (to insert) and the associated SI. This model is adapted from the minimal microphysical framework presented in Bernd Kärcher et al., 2015 (https://doi.org/10.1002/2015JD023491).

In [1]:
# required packages

import numpy as np
import pandas as pd
import xarray as xr

import scipy    
from scipy import interpolate
from scipy.special import erfc
from scipy.special import erf
from scipy.integrate import quad
from scipy.optimize import fsolve

# define relevant constants and temperature-space

k_b = 1.38e-23 # Boltzmann's constant / J/K/mol
nu = (18.0e-3/6.022e23)/1000 # volume of a supercooled water molecule / m^3 
R_g = 8.3145 # global gas constant
v_thermal = np.sqrt((8 * k_b * 298.15)/(np.pi * nu * 1000)) # mean thermal velocity of water molecule / m/s
T = np.linspace(220, 245, 10000) # plume temperature / K

# various functions

def e_w(T):
    
    # returns the saturation vapour pressure above liquid water

    A = 54.842763
    B = 6763.22
    C = 4.21
    D = 0.000367
    E = 0.0415
    F = 218.8
    G = 53.878
    H = 1331.22
    I = 9.44523
    J = 0.014025
    
    return np.exp(A - B/T - C * np.log(T) + D*T + np.tanh(E * (T - F)) * (G - H/T - I * np.log(T) + J*T)) 

def p_i_0(T):
    
    # returns the saturation vapour pressure above ice
    
    return np.exp(9.550426 - 5723.265/T + 3.53068 * np.log(T) - 0.00728332*T) 

def pressure(z):
    
    # returns the pressure (Pa) at a given altitude

    if z < 11000:
        return 101325 * (1 - 2.25577e-5 * z) ** 5.25589
    else:
        return 22632 * np.exp(-1.57689e-4 * (z - 11000))

def cumulative_gaussian(Dp, Nt0, Dp_mean0, sigma_g0):
    
    # returns cumulative number concentration for a lognormal size distribution

    distribution = Nt0 - Nt0 * (0.5 + 0.5 * erf((np.log(Dp) - np.log(Dp_mean0))/((2)**(0.5) * np.log(sigma_g0))))
    return distribution 

def r_act(kappa_mode, s_w, T):

    # returns critical activation radii
    
    db = da.interp(Kappa = kappa_mode, Temperature = T)
    d_act_interp = []

    for index, subarray in enumerate(db.T):
                
        f = interpolate.interp1d(subarray, Dd, fill_value = 'extrapolate')
        interpolated_y = f(s_w[index] + 1)
            
        d_act_interp.append(interpolated_y)
      
    return np.array(d_act_interp)/2

class contrail_mixing_properties:
    
    def __init__(self, G, T_a, T_e, tau_m, beta, N_0, mean_contrail_height):
        
        self.G = G
        self.T_a = T_a
        self.T_e = T_e
        self.T = T
        self.beta = beta
        self.tau_m = tau_m
        self.N_0 = N_0
        self.mean_contrail_height = mean_contrail_height
    
    def p_mw(self, T):

        # returns partial pressure of water vapour in the contrail mixing plume
        
        p_wa = p_i_0(T_a) 
                    
        return p_wa + self.G * (T - self.T_a) 
    
    def S_mw(self, T):

        # returns water saturation ratio in the contrail mixing plume
        
        return self.p_mw(T)/e_w(T)
    
    def s_w(self, T):

        # returns water supersaturation in the contrail mixing plume
        
        return self.S_mw(T) - 1
    
    def T_x_index(self, T):

        # returns index of intersection of saturation vapour pressure above water and the contrail mixing line
    
        T_x = fsolve(lambda T: self.p_mw(T) - e_w(T), x0 = self.T_e)

        return np.argmin(np.abs(T - T_x))
    
    def T_n_index(self, T):

        # returns the index corresponding to the maximum plume supersaturation
        
        return np.argmax(self.S_mw(T))
    
    def D(self, T):

        # returns the plume dilution factor
        
        t = self.tau_m * ((self.T_e - self.T_a) / (T - self.T_a)) ** (1/self.beta)
        D_array = np.where(t > self.tau_m, (self.tau_m/t) ** self.beta, 1)
        return D_array
    
    def time_elapsed(self, T):

        # returns the time elapsed from exhaust
        
        t = self.tau_m * ((self.T_e - self.T_a) / (T - self.T_a)) ** (1/self.beta)
        return t

    def P_w(self, T):

        # returns the water (super)saturation production rate in the absence of particles 
    
        dT_dt = - self.beta * (self.T_e - self.T_a)/self.tau_m * self.D(T) ** (1 + 1/self.beta)
        d_S_mw_dT = np.gradient(self.S_mw(T), T)
        return d_S_mw_dT * dT_dt
    
    def mean_contrail_air_density(self, T):

        # returns the mean air density in the contrail
        
        mean_contrail_air_pressure = pressure(self.mean_contrail_height)         
        return (mean_contrail_air_pressure * 28.96e-3) / (R_g * T) # kg/m^3    
    
class particle_properties:
    
    def __init__(self, contrail, kappa, GMR, GSD, EI = None, n_init = None, sort = None):
        
        self.kappa = kappa
        self.GMR = GMR
        self.GSD = GSD
        self.EI = EI
        self.n_init = n_init
        self.sort = sort
        self.r_act = r_act_dict[f'{self.kappa}']
        self.contrail = contrail
    
    def n_present(self):

        # returns the number of particles of a given mode that are present in the mixing plume
        
        if self.n_init == None:         
            return contrail.mean_contrail_air_density(T) * self.EI * contrail.D(T) / contrail.N_0
        else:
            return contrail.T_a/T * (1 - contrail.D(T)) * self.n_init 
                                    
    def n_available(self):

        # returns the number of particles of a given mode that can be activated in the mixing plume

        return cumulative_gaussian(2 * self.r_act, self.n_present(), self.GMR * 2, self.GSD) 

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


The below cell imports data for the critical supersaturation as a function of temperature, dry particle diameter and hygroscopicity parameter (this is for the user to generate). Also, the contrail object is initialised subject to the defined properties.

In [2]:
# import 𝜅-Köhler data array in file format Sc = Sc(Dd, T, kappa)

# da = xr.open_dataarray("filepath") 

Dd = da.Dry_Particle_Diameter

# define contrail properties

mean_contrail_height = 10919.85 # mean contrail altitude / m
G = 1.64 # gradient of average contrail mixing line / Pa/K
T_a = 215 # ambient temperature / K
T_e = 600 # exhaust temperature / K
tau_m = 10e-3 # timescale over which contrail mixing is unaffected by entrainment / s
beta = 0.9 # plume dilution parameter
N_0 = 60 # mass-based mixing factor

# initialise contrail object

contrail = contrail_mixing_properties(G, T_a, T_e, tau_m, beta, N_0, mean_contrail_height) 

# determine critical activation radii

r_act_dict = {}

for i in np.array(da.Kappa):
    key = f'{i}' 
    value = r_act(i, contrail.s_w(T), T)
    r_act_dict[key] = value  

/opt/anaconda3/envs/contrail_camera/lib/python3.8/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/opt/anaconda3/envs/contrail_camera/lib/python3.8/site-packages/xarray/core/missing.py:563: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imax = index.get_loc(maxval, method="nearest")
/opt/anaconda3/envs/contrail_camera/lib/python3.8/site-packages/xarray/core/missing.py:562: FutureWarning: Passing method to Float64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.
  imin = index.get_loc(minval, method="nearest")
/opt/anaconda3/envs/contrail_camera/lib/python3.8/site-packages/xarray/core/missing.py:563: FutureWarning: Passi

The below cell allows for the apparent emission index of ice and maximum plume supersaturation to be estimated. The particle properties should be changed within the function.

In [3]:
# estimate apparent emission index of ice

def f_K15(EI_nvPM, EI_vPM): 

    # define the particle modes using their hygroscopicity, GMD, GSD and EI (or initial number concentration)
            
    particle_modes = {
        "nvPM": particle_properties(contrail, 0.005, 17.5e-9, 2.0, EI = EI_nvPM, sort = "nvPM"), 
        "vPM": particle_properties(contrail, 0.5, 1.25e-9, 1.3, EI = EI_vPM, sort = "vPM"), 
        "ambient": particle_properties(contrail, 0.5, 15e-9, 2.2, n_init = 600e6, sort = "ambient"),                              
    }
    
    """PONSONBY TAU_ACT""" 
    
    weighted_sum = 0
    total_weights = 0
    total_activated = 0
    total_ambient = 0
    
    for key, obj in particle_modes.items():
        
        weighted_sum += obj.n_available() * obj.r_act
        total_weights += obj.n_available() 
        total_activated += obj.n_available()  
    
        if obj.sort == "ambient":
            total_ambient += obj.n_available() 
    
    r_act_total_mean = weighted_sum/total_weights # mean activation radius / m

    # identify various variables for later calculating the water (super)saturation depletion rate due to particle activation and droplet growth
    
    ln_n_w = np.log(total_activated)
    d_ln_n_w_d_s_w = np.gradient(ln_n_w, contrail.s_w(T))
    
    n_w_sat = e_w(T)/(k_b * T) # water vapour concentration at saturation / m^-3
    v_thermal = np.sqrt((8 * R_g * T)/(np.pi * 18.0e-3)) # mean thermal velocity of water molecule based on molar mass / m/s 
    
    ambient_pressure = pressure(contrail.mean_contrail_height) # ambient pressure / Pa
    D = 0.211 * (T/273.15) ** 1.94 * (101325/ambient_pressure) * 1e-4 # water diffusivity / m^2/s
    tau_act = 1/(contrail.P_w(T) * d_ln_n_w_d_s_w) # activation timescale / s
    b_2 = ((1 * v_thermal)/(4 * D))
    a = (2 * 0.072 * 18e-3)/(R_g * 1e3 * T)
    b_1 = (nu * v_thermal * contrail.s_w(T) * n_w_sat)/4
    
    # identify the water (super)saturation depletion rate due to particle activation and droplet growth
    
    delta = b_2 * r_act_total_mean 
    tau_g = (1 + delta)/(b_1/r_act_total_mean) # growth timescale / s
    kappa = (2 * delta) / (1 + delta) * tau_act/tau_g 
    f_kappa = np.sqrt(np.pi * kappa) * np.exp(1/kappa) * erfc(1/np.sqrt(kappa))
    
    R_w = (4 * np.pi)/(nu * n_w_sat) * b_1/(b_2 ** 2) * (delta ** 2)/(1 + delta) *  (1 - 1/(delta ** 2) + 1/(delta ** 2) * (((1 + delta) ** 2)/2 + 1/kappa) * f_kappa) 
    n_2_w = contrail.P_w(T)/R_w # required number concentration of droplets to quench plume supersaturation
    
    # identify intersection between "n_2_w" and "total_activated"
    
    f = np.abs(np.log(total_activated) - np.log(n_2_w))   
    n_0_index = np.argmax(f ** 7 < 1e-7)
    n_0 = n_2_w[n_0_index]
    
    # identify the apparent emission index of ice crystals
        
    AEIi = (n_0 * contrail.N_0) / (contrail.D(T)[n_0_index] * contrail.mean_contrail_air_density(T[n_0_index]))

    return AEIi, contrail.s_w(T)[n_0_index] # return predicted AEI_ice and maximum plume supersaturation

f_K15_vectorized = np.vectorize(f_K15)